In [14]:
import pandas as pd
import datetime
import numpy as np # linear algebra
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [15]:
num = range(7)

In [24]:
final_data = pd.read_csv("https://raw.githubusercontent.com/Yifeng-T/BIOS823_Final/main/Raw_data/final_data.csv")

In [26]:
final_data_list = {}

In [27]:
for i in num:
    final_data_test = final_data.copy(deep=True)
    j = i + 1
    final_data_test["lag"+str(j+0)+"c"] = final_data_test.groupby("State").Daily_Case.shift(j+0)
    final_data_test["lag"+str(j+1)+"c"] = final_data_test.groupby("State").Daily_Case.shift(j+1)
    final_data_test["lag"+str(j+2)+"c"] = final_data_test.groupby("State").Daily_Case.shift(j+2)
    final_data_test["lag"+str(j+3)+"c"] = final_data_test.groupby("State").Daily_Case.shift(j+3)
    final_data_test["lag"+str(j+4)+"c"] = final_data_test.groupby("State").Daily_Case.shift(j+4)
    final_data_test["lag"+str(j+5)+"c"] = final_data_test.groupby("State").Daily_Case.shift(j+5)
    final_data_test["lag"+str(j+6)+"c"] = final_data_test.groupby("State").Daily_Case.shift(j+6)
    
    final_data_test["lag"+str(j+0)+"d"] = final_data_test.groupby("State").Daily_Death.shift(j+0)
    final_data_test["lag"+str(j+1)+"d"] = final_data_test.groupby("State").Daily_Death.shift(j+1)
    final_data_test["lag"+str(j+2)+"d"] = final_data_test.groupby("State").Daily_Death.shift(j+2)
    final_data_test["lag"+str(j+3)+"d"] = final_data_test.groupby("State").Daily_Death.shift(j+3)
    final_data_test["lag"+str(j+4)+"d"] = final_data_test.groupby("State").Daily_Death.shift(j+4)
    final_data_test["lag"+str(j+5)+"d"] = final_data_test.groupby("State").Daily_Death.shift(j+5)
    final_data_test["lag"+str(j+6)+"d"] = final_data_test.groupby("State").Daily_Death.shift(j+6)
    
    final_data_test["lag"+str(j+0)+"t"] = final_data_test.groupby("State").tests_combined_total.shift(j+0)
    final_data_test["lag"+str(j+1)+"t"] = final_data_test.groupby("State").tests_combined_total.shift(j+1)
    final_data_test["lag"+str(j+2)+"t"] = final_data_test.groupby("State").tests_combined_total.shift(j+2)
    final_data_test["lag"+str(j+3)+"t"] = final_data_test.groupby("State").tests_combined_total.shift(j+3)
    final_data_test["lag"+str(j+4)+"t"] = final_data_test.groupby("State").tests_combined_total.shift(j+4)
    final_data_test["lag"+str(j+5)+"t"] = final_data_test.groupby("State").tests_combined_total.shift(j+5)
    final_data_test["lag"+str(j+6)+"t"] = final_data_test.groupby("State").tests_combined_total.shift(j+6)
    
    final_data_test["lag"+str(j+0)+"v"] = final_data_test.groupby("State").People_Fully_Vaccinated.shift(j+0)
    final_data_test["lag"+str(j+1)+"v"] = final_data_test.groupby("State").People_Fully_Vaccinated.shift(j+1)
    final_data_test["lag"+str(j+2)+"v"] = final_data_test.groupby("State").People_Fully_Vaccinated.shift(j+2)
    final_data_test["lag"+str(j+3)+"v"] = final_data_test.groupby("State").People_Fully_Vaccinated.shift(j+3)
    final_data_test["lag"+str(j+4)+"v"] = final_data_test.groupby("State").People_Fully_Vaccinated.shift(j+4)
    final_data_test["lag"+str(j+5)+"v"] = final_data_test.groupby("State").People_Fully_Vaccinated.shift(j+5)
    final_data_test["lag"+str(j+6)+"v"] = final_data_test.groupby("State").People_Fully_Vaccinated.shift(j+6)
    
    
    final_data_test = final_data_test.dropna().copy(deep = True)
    
    final_data_test['Fully_reopen']        = final_data_test['Fully_reopen'].astype(object)
    final_data_test['Mask_Mandate']        = final_data_test['Mask_Mandate'].astype(object)
    final_data_test['Vaccination_or_test'] = final_data_test['Vaccination_or_test'].astype(object)
    final_data_test['State']               = final_data_test['State'].astype(object)
    final_data_test['Region']              = final_data_test['Region'].astype(object)
    final_data_test['Division']            = final_data_test['Division'].astype(object)
    
    final_data_test = final_data_test.drop(columns=['People_Fully_Vaccinated', 'tests_combined_total']).copy(deep = True)

    final_data_list["final_data_"+str(j)] = final_data_test
    
    

In [49]:
data_list = []
ran = list(final_data_list.keys())
for i in ran:
    data = final_data_list[i]
    data = data.drop(["Date", "State", "State"], 1)
    data = pd.get_dummies(data)
    data_list.append(data)
len(data_list)

7

In [28]:
# data = final_data_list["final_data_1"]
# data = data.drop(["Date","State","state"], 1)
# data.columns

Index(['Region', 'Division', 'Daily_Death', 'Daily_Case',
       'Above_Poverty_Level_Population', 'No_Health_Insurance_Population',
       'population', 'Median_Income', '65_Years_or_More_Population',
       '25_Years_or_More_Population', 'Population_Density',
       'Bachelors_Degree_or_Higher_Population', 'Fully_reopen', 'Mask_Mandate',
       'Vaccination_or_test', 'Ratio of people per hospital employee',
       'Ratio of people per hospital', 'gdp_2020', 'employed_population',
       'lag1c', 'lag2c', 'lag3c', 'lag4c', 'lag5c', 'lag6c', 'lag7c', 'lag1d',
       'lag2d', 'lag3d', 'lag4d', 'lag5d', 'lag6d', 'lag7d', 'lag1t', 'lag2t',
       'lag3t', 'lag4t', 'lag5t', 'lag6t', 'lag7t', 'lag1v', 'lag2v', 'lag3v',
       'lag4v', 'lag5v', 'lag6v', 'lag7v'],
      dtype='object')

In [29]:
# data = pd.get_dummies(data)
# data.columns

Index(['Daily_Death', 'Daily_Case', 'Above_Poverty_Level_Population',
       'No_Health_Insurance_Population', 'population', 'Median_Income',
       '65_Years_or_More_Population', '25_Years_or_More_Population',
       'Population_Density', 'Bachelors_Degree_or_Higher_Population',
       'Ratio of people per hospital employee', 'Ratio of people per hospital',
       'gdp_2020', 'employed_population', 'lag1c', 'lag2c', 'lag3c', 'lag4c',
       'lag5c', 'lag6c', 'lag7c', 'lag1d', 'lag2d', 'lag3d', 'lag4d', 'lag5d',
       'lag6d', 'lag7d', 'lag1t', 'lag2t', 'lag3t', 'lag4t', 'lag5t', 'lag6t',
       'lag7t', 'lag1v', 'lag2v', 'lag3v', 'lag4v', 'lag5v', 'lag6v', 'lag7v',
       'Region_Midwest', 'Region_Northeast', 'Region_South', 'Region_West',
       'Division_East North Central', 'Division_East South Central',
       'Division_Middle Atlantic', 'Division_Mountain', 'Division_New England',
       'Division_Pacific', 'Division_South Atlantic',
       'Division_West North Central', 'Divisi

In [53]:
train_and_test = []

for i in data_list:
    train, test = train_test_split(i, test_size=0.2, random_state=42)
    item = [train, test]
    train_and_test.append(item)
len(train_and_test[0][0])

10980

In [31]:
# len(train)

10980

In [54]:
y_train = []
X_train = []
y_test = []
X_test = []
for i in train_and_test:
    y_train_item = i[0]["Daily_Death"]
    y_train.append(y_train_item)
    
    X_train_item = i[0].drop(["Daily_Case","Daily_Death"], 1)
    X_train.append(X_train_item)
    
    y_test_item = i[1]["Daily_Death"]
    y_test.append(y_test_item)
    
    X_test_item = i[1].drop(["Daily_Case","Daily_Death"], 1)
    X_test.append(X_test_item)

In [32]:
# y_train = train["Daily_Death"]
# X_train = train.drop(["Daily_Case","Daily_Death"],1)
# y_test = test["Daily_Death"]
# X_test = test.drop(["Daily_Case","Daily_Death"],1)

In [62]:
dtrain_list = []
dtest_list = []

for i in range(0, len(y_test)):
    dtrain_item = xgb.DMatrix(X_train[i], label=y_train[i])
    dtrain_list.append(dtrain_item)
    
    dtest_item = xgb.DMatrix(X_test[i], label=y_test[i])
    dtest_list.append(dtest_item)
#len(dtest_list)


7

In [33]:
# dtrain = xgb.DMatrix(X_train, label=y_train)
# dtest = xgb.DMatrix(X_test, label=y_test)

In [58]:
import numpy as np
mae_baseline = []
for i in range(0, len(y_train)):
    mean_train = np.mean(y_train[i])
    baseline_predictions = np.ones(y_test[i].shape) * mean_train
    mae_baseline_item = mean_absolute_error(y_test[i], baseline_predictions)
    mae_baseline.append(mae_baseline_item)
mae_baseline
    
    

[21.25133069860184,
 20.499692367726137,
 20.02339119723761,
 19.40432902898586,
 18.657911562311725,
 19.410352036687346,
 19.112148862090624]

In [34]:
# """Calculate Baseline MAE"""
# import numpy as np
# # "Learn" the mean from the training data
# mean_train = np.mean(y_train)
# # Get predictions on the test set
# baseline_predictions = np.ones(y_test.shape) * mean_train
# # Compute MAE
# mae_baseline = mean_absolute_error(y_test, baseline_predictions)
# print("Baseline MAE is {:.2f}".format(mae_baseline))

Baseline MAE is 21.25


In [59]:
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:linear',
}


In [60]:
params['eval_metric'] = "mae"

In [61]:
num_boost_round = 999

In [65]:
model_all = []
for i in range(0, 7):
    model = xgb.train(
        params,
        dtrain_list[i],
        num_boost_round=num_boost_round,
        evals=[(dtest_list[i], "Test")],
        early_stopping_rounds=10)
    model_all.append(model.best_iteration+1)
len(model_all)

[01:54:57] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[0]	Test-mae:14.39347
Will train until Test-mae hasn't improved in 10 rounds.
[1]	Test-mae:11.73385
[2]	Test-mae:9.99873
[3]	Test-mae:9.11913
[4]	Test-mae:8.62105
[5]	Test-mae:8.29998
[6]	Test-mae:8.11602
[7]	Test-mae:8.03250
[8]	Test-mae:7.97439
[9]	Test-mae:7.94127
[10]	Test-mae:7.88207
[11]	Test-mae:7.89336
[12]	Test-mae:7.88628
[13]	Test-mae:7.89145
[14]	Test-mae:7.87976
[15]	Test-mae:7.87561
[16]	Test-mae:7.87769
[17]	Test-mae:7.87572
[18]	Test-mae:7.85816
[19]	Test-mae:7.86397
[20]	Test-mae:7.84258
[21]	Test-mae:7.85467
[22]	Test-mae:7.86624
[23]	Test-mae:7.85501
[24]	Test-mae:7.85639
[25]	Test-mae:7.87216
[26]	Test-mae:7.89367
[27]	Test-mae:7.88925
[28]	Test-mae:7.89161
[29]	Test-mae:7.87656
[30]	Test-mae:7.87408
Stopping. Best iteration:
[20]	Test-mae:7.84258

[01:54:57] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecat

7

In [66]:
# model = xgb.train(
#     params,
#     dtrain,
#     num_boost_round=num_boost_round,
#     evals=[(dtest, "Test")],
#     early_stopping_rounds=10
# )
# print("Best MAE: {:.2f} with {} rounds".format(
#                  model.best_score,
#                  model.best_iteration+1))


In [39]:
# You can try wider intervals with a larger step between
# each value and then narrow it down. Here after several
# # iteration I found that the optimal value was in the
# # following ranges.
# gridsearch_params = [
#     (max_depth, min_child_weight)
#     for max_depth in range(0,12)
#     for min_child_weight in range(0,8)
# ]

In [67]:
max_depth_list=[]
min_child_weight_list = []

for i in range(0, 7):
    gridsearch_params = [
        (max_depth, min_child_weight)
        for max_depth in range(0,10)
        for min_child_weight in range(0,8)]
    min_mae = float("Inf")
    best_params = None
    for max_depth, min_child_weight in gridsearch_params:
        print("CV with max_depth={}, min_child_weight={}".format(max_depth, min_child_weight))
        
        # Update our parameters
        params['max_depth'] = max_depth
        params['min_child_weight'] = min_child_weight
        # Run CV
        cv_results = xgb.cv(
            params,
            dtrain_list[i],
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics={'mae'},
            early_stopping_rounds=10)
        # Update best MAE
        mean_mae = cv_results['test-mae-mean'].min()
        boost_rounds = cv_results['test-mae-mean'].argmin()
        #print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
        if mean_mae < min_mae:
            min_mae = mean_mae
            best_params = (max_depth,min_child_weight)
    max_depth_list.append(best_params[0])
    min_child_weight_list.append(best_params[1])
            
        
    
    

CV with max_depth=0, min_child_weight=0
[02:04:03] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:04:03] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:04:03] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:04:03] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:04:03] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
CV with max_depth=0, min_child_weight=1
[02:04:03] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:04:03] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:04:03] WARNING: /workspace/

CV with max_depth=1, min_child_weight=5
[02:04:14] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:04:14] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:04:14] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:04:14] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:04:14] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
CV with max_depth=1, min_child_weight=6
[02:04:16] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:04:16] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:04:16] WARNING: /workspace/

CV with max_depth=3, min_child_weight=2
[02:04:46] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:04:46] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:04:46] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:04:46] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:04:46] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
CV with max_depth=3, min_child_weight=3
[02:04:48] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:04:48] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:04:48] WARNING: /workspace/

[02:05:13] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
CV with max_depth=4, min_child_weight=7
[02:05:15] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:05:15] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:05:15] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:05:15] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:05:15] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
CV with max_depth=5, min_child_weight=0
[02:05:17] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:05:17] WARNING: /workspace/

CV with max_depth=6, min_child_weight=3
[02:05:44] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:05:44] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:05:44] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:05:44] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:05:44] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
CV with max_depth=6, min_child_weight=4
[02:05:47] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:05:47] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:05:47] WARNING: /workspace/

...

CV with max_depth=2, min_child_weight=3
[02:22:50] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:22:50] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:22:50] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:22:50] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:22:50] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
CV with max_depth=2, min_child_weight=4
[02:22:53] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:22:53] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:22:53] WARNING: /workspace/

CV with max_depth=4, min_child_weight=0
[02:23:20] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:23:20] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:23:20] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:23:20] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:23:20] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
CV with max_depth=4, min_child_weight=1
[02:23:22] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:23:22] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:23:22] WARNING: /workspace/

CV with max_depth=5, min_child_weight=5
[02:23:40] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:23:40] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:23:40] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:23:40] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:23:40] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
CV with max_depth=5, min_child_weight=6
[02:23:42] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:23:42] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:23:42] WARNING: /workspace/

CV with max_depth=7, min_child_weight=2
[02:24:01] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:24:01] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:24:01] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:24:01] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:24:01] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
CV with max_depth=7, min_child_weight=3
[02:24:03] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:24:03] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:24:03] WARNING: /workspace/

[02:24:25] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
CV with max_depth=8, min_child_weight=7
[02:24:27] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:24:27] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:24:27] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:24:27] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:24:27] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
CV with max_depth=9, min_child_weight=0
[02:24:29] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:24:29] WARNING: /workspace/

In [69]:
max_depth_list

[7, 8, 7, 8, 7, 8, 9]

In [70]:
min_child_weight_list

[6, 7, 0, 6, 7, 0, 7]

In [68]:
# You can try wider intervals with a larger step between
# each value and then narrow it down. Here after several
# iteration I found that the optimal value was in the
# following ranges.



# gridsearch_params = [
#     (max_depth, min_child_weight)
#     for max_depth in range(0,12)
#     for min_child_weight in range(0,8)
# ]

# # Define initial best params and MAE
# min_mae = float("Inf")
# best_params = None
# for max_depth, min_child_weight in gridsearch_params:
#     print("CV with max_depth={}, min_child_weight={}".format(
#                              max_depth,
#                              min_child_weight))
#     # Update our parameters
#     params['max_depth'] = max_depth
#     params['min_child_weight'] = min_child_weight
#     # Run CV
#     cv_results = xgb.cv(
#         params,
#         dtrain,
#         num_boost_round=num_boost_round,
#         seed=42,
#         nfold=5,
#         metrics={'mae'},
#         early_stopping_rounds=10
#     )
#     # Update best MAE
#     mean_mae = cv_results['test-mae-mean'].min()
#     boost_rounds = cv_results['test-mae-mean'].argmin()
#     print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
#     if mean_mae < min_mae:
#         min_mae = mean_mae
#         best_params = (max_depth,min_child_weight)
# print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

In [71]:
subsample_list = []
colsample_list = []

for i in range(0,7):
    gridsearch_params = [
        (subsample, colsample)
        for subsample in [i/10. for i in range(7,11)]
        for colsample in [i/10. for i in range(7,11)]]

    min_mae = float("Inf")
    best_params = None
    # We start by the largest values and go down to the smallest
    for subsample, colsample in reversed(gridsearch_params):
        print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
        # We update our parameters
        params['subsample'] = subsample
        params['colsample_bytree'] = colsample
        # Run CV
        cv_results = xgb.cv(
            params,
            dtrain_list[i],
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics={'mae'},
            early_stopping_rounds=10)
        # Update best score
        mean_mae = cv_results['test-mae-mean'].min()
        boost_rounds = cv_results['test-mae-mean'].argmin()
        #print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
        if mean_mae < min_mae:
            min_mae = mean_mae
            best_params = (subsample,colsample)
    subsample_list.append(best_params[0])
    colsample_list.append(best_params[1])

CV with subsample=1.0, colsample=1.0
[02:31:23] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:31:23] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:31:23] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:31:23] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:31:23] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
CV with subsample=1.0, colsample=0.9
[02:31:26] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:31:26] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:31:26] WARNING: /workspace/src/ob

[02:31:51] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
CV with subsample=0.7, colsample=0.9
[02:31:53] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:31:53] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:31:53] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:31:53] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:31:53] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
CV with subsample=0.7, colsample=0.8
[02:31:55] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:31:55] WARNING: /workspace/src/ob

[02:32:18] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
CV with subsample=0.8, colsample=0.8
[02:32:20] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:32:20] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:32:20] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:32:21] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:32:21] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
CV with subsample=0.8, colsample=0.7
[02:32:22] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:32:22] WARNING: /workspace/src/ob

[02:33:28] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
CV with subsample=0.9, colsample=0.7
[02:33:30] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:33:30] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:33:30] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:33:30] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:33:30] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
CV with subsample=0.8, colsample=1.0
[02:33:32] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:33:32] WARNING: /workspace/src/ob

CV with subsample=0.9, colsample=1.0
[02:33:58] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:33:58] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:33:58] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:33:58] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:33:58] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
CV with subsample=0.9, colsample=0.9
[02:34:01] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:34:01] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:34:01] WARNING: /workspace/src/ob

[02:34:24] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
CV with subsample=1.0, colsample=0.9
[02:34:26] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:34:26] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:34:26] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:34:26] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:34:26] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
CV with subsample=1.0, colsample=0.8
[02:34:28] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:34:28] WARNING: /workspace/src/ob

[02:34:52] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
CV with subsample=0.7, colsample=0.8
[02:34:54] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:34:54] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:34:54] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:34:54] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:34:54] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
CV with subsample=0.7, colsample=0.7
[02:34:57] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:34:57] WARNING: /workspace/src/ob

[02:35:24] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
CV with subsample=0.8, colsample=0.7
[02:35:26] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:35:27] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:35:27] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:35:27] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:35:27] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
CV with subsample=0.7, colsample=1.0
[02:35:29] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:35:29] WARNING: /workspace/src/ob

CV with subsample=0.8, colsample=1.0
[02:35:56] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:35:56] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:35:56] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:35:56] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:35:56] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
CV with subsample=0.8, colsample=0.9
[02:35:58] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:35:58] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:35:58] WARNING: /workspace/src/ob

In [73]:
subsample_list

[0.8, 0.9, 0.8, 0.9, 1.0, 1.0, 1.0]

In [74]:
colsample_list

[1.0, 0.9, 0.9, 1.0, 0.8, 1.0, 1.0]

In [ ]:
# gridsearch_params = [
#     (subsample, colsample)
#     for subsample in [i/10. for i in range(7,11)]
#     for colsample in [i/10. for i in range(7,11)]
# ]

# min_mae = float("Inf")
# best_params = None
# # We start by the largest values and go down to the smallest
# for subsample, colsample in reversed(gridsearch_params):
#     print("CV with subsample={}, colsample={}".format(
#                              subsample,
#                              colsample))
#     # We update our parameters
#     params['subsample'] = subsample
#     params['colsample_bytree'] = colsample
#     # Run CV
#     cv_results = xgb.cv(
#         params,
#         dtrain,
#         num_boost_round=num_boost_round,
#         seed=42,
#         nfold=5,
#         metrics={'mae'},
#         early_stopping_rounds=10
#     )
#     # Update best score
#     mean_mae = cv_results['test-mae-mean'].min()
#     boost_rounds = cv_results['test-mae-mean'].argmin()
#     print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
#     if mean_mae < min_mae:
#         min_mae = mean_mae
#         best_params = (subsample,colsample)
# print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

In [75]:
eta_list = []
for i in range(0,7):
    min_mae = float("Inf")
    best_params = None
    for eta in [.3, .2, .1, .05, .01, .005]:
        print("CV with eta={}".format(eta))
        # We update our parameters
        params['eta'] = eta
        # Run and time CV
        cv_results = xgb.cv(params,
                            dtrain_list[i],
                            num_boost_round=num_boost_round,
                            seed=42,
                            nfold=5,
                            metrics=['mae'],
                            early_stopping_rounds=10)
        # Update best score
        mean_mae = cv_results['test-mae-mean'].min()
        boost_rounds = cv_results['test-mae-mean'].argmin()
        #print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
        if mean_mae < min_mae:
            min_mae = mean_mae
            best_params = eta
    eta_list.append(best_params)

CV with eta=0.3
[02:39:28] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:39:28] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:39:28] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:39:28] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:39:28] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
CV with eta=0.2
[02:39:30] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:39:30] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:39:30] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear 

CV with eta=0.2
[02:43:50] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:43:50] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:43:50] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:43:50] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:43:50] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
CV with eta=0.1
[02:43:53] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:43:53] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:43:53] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear 

CV with eta=0.1
[02:48:36] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:48:36] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:48:36] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:48:36] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:48:36] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
CV with eta=0.05
[02:48:40] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:48:41] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:48:41] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear

CV with eta=0.05
[02:53:28] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:53:28] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:53:28] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:53:28] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:53:28] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
CV with eta=0.01
[02:53:35] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:53:35] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[02:53:36] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linea

In [ ]:
# min_mae = float("Inf")
# best_params = None
# for eta in [.3, .2, .1, .05, .01, .005]:
#     print("CV with eta={}".format(eta))
#     # We update our parameters
#     params['eta'] = eta
#     # Run and time CV
#     %time cv_results = xgb.cv(
#             params,
#             dtrain,
#             num_boost_round=num_boost_round,
#             seed=42,
#             nfold=5,
#             metrics=['mae'],
#             early_stopping_rounds=10
#           )
#     # Update best score
#     mean_mae = cv_results['test-mae-mean'].min()
#     boost_rounds = cv_results['test-mae-mean'].argmin()
#     print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
#     if mean_mae < min_mae:
#         min_mae = mean_mae
#         best_params = eta
# print("Best params: {}, MAE: {}".format(best_params, min_mae))


In [76]:
eta_list

[0.01, 0.005, 0.005, 0.005, 0.005, 0.005, 0.01]

In [83]:
model_list = []
params_list = []

for i in range(0,7):
    params = {'colsample_bytree': colsample_list[i], 
              'eta': eta_list[i],
              'eval_metric': 'mae',
              'max_depth': max_depth_list[i],
              'min_child_weight': min_child_weight_list[i],
              'objective': 'reg:linear',
              'subsample': subsample_list[i]}
    params_list.append(params)

In [98]:
params_list

[{'colsample_bytree': 1.0,
  'eta': 0.01,
  'eval_metric': 'mae',
  'max_depth': 7,
  'min_child_weight': 6,
  'objective': 'reg:linear',
  'subsample': 0.8},
 {'colsample_bytree': 0.9,
  'eta': 0.005,
  'eval_metric': 'mae',
  'max_depth': 8,
  'min_child_weight': 7,
  'objective': 'reg:linear',
  'subsample': 0.9},
 {'colsample_bytree': 0.9,
  'eta': 0.005,
  'eval_metric': 'mae',
  'max_depth': 7,
  'min_child_weight': 0,
  'objective': 'reg:linear',
  'subsample': 0.8},
 {'colsample_bytree': 1.0,
  'eta': 0.005,
  'eval_metric': 'mae',
  'max_depth': 8,
  'min_child_weight': 6,
  'objective': 'reg:linear',
  'subsample': 0.9},
 {'colsample_bytree': 0.8,
  'eta': 0.005,
  'eval_metric': 'mae',
  'max_depth': 7,
  'min_child_weight': 7,
  'objective': 'reg:linear',
  'subsample': 1.0},
 {'colsample_bytree': 1.0,
  'eta': 0.005,
  'eval_metric': 'mae',
  'max_depth': 8,
  'min_child_weight': 0,
  'objective': 'reg:linear',
  'subsample': 1.0},
 {'colsample_bytree': 1.0,
  'eta': 0.01,

In [99]:
p = [{'colsample_bytree': 1.0,
  'eta': 0.01,
  'eval_metric': 'mae',
  'max_depth': 7,
  'min_child_weight': 6,
  'objective': 'reg:linear',
  'subsample': 0.8},
 {'colsample_bytree': 0.9,
  'eta': 0.005,
  'eval_metric': 'mae',
  'max_depth': 8,
  'min_child_weight': 7,
  'objective': 'reg:linear',
  'subsample': 0.9},
 {'colsample_bytree': 0.9,
  'eta': 0.005,
  'eval_metric': 'mae',
  'max_depth': 7,
  'min_child_weight': 0,
  'objective': 'reg:linear',
  'subsample': 0.8},
 {'colsample_bytree': 1.0,
  'eta': 0.005,
  'eval_metric': 'mae',
  'max_depth': 8,
  'min_child_weight': 6,
  'objective': 'reg:linear',
  'subsample': 0.9},
 {'colsample_bytree': 0.8,
  'eta': 0.005,
  'eval_metric': 'mae',
  'max_depth': 7,
  'min_child_weight': 7,
  'objective': 'reg:linear',
  'subsample': 1.0},
 {'colsample_bytree': 1.0,
  'eta': 0.005,
  'eval_metric': 'mae',
  'max_depth': 8,
  'min_child_weight': 0,
  'objective': 'reg:linear',
  'subsample': 1.0},
 {'colsample_bytree': 1.0,
  'eta': 0.01,
  'eval_metric': 'mae',
  'max_depth': 9,
  'min_child_weight': 7,
  'objective': 'reg:linear',
  'subsample': 1.0}]

In [100]:
p

[{'colsample_bytree': 1.0,
  'eta': 0.01,
  'eval_metric': 'mae',
  'max_depth': 7,
  'min_child_weight': 6,
  'objective': 'reg:linear',
  'subsample': 0.8},
 {'colsample_bytree': 0.9,
  'eta': 0.005,
  'eval_metric': 'mae',
  'max_depth': 8,
  'min_child_weight': 7,
  'objective': 'reg:linear',
  'subsample': 0.9},
 {'colsample_bytree': 0.9,
  'eta': 0.005,
  'eval_metric': 'mae',
  'max_depth': 7,
  'min_child_weight': 0,
  'objective': 'reg:linear',
  'subsample': 0.8},
 {'colsample_bytree': 1.0,
  'eta': 0.005,
  'eval_metric': 'mae',
  'max_depth': 8,
  'min_child_weight': 6,
  'objective': 'reg:linear',
  'subsample': 0.9},
 {'colsample_bytree': 0.8,
  'eta': 0.005,
  'eval_metric': 'mae',
  'max_depth': 7,
  'min_child_weight': 7,
  'objective': 'reg:linear',
  'subsample': 1.0},
 {'colsample_bytree': 1.0,
  'eta': 0.005,
  'eval_metric': 'mae',
  'max_depth': 8,
  'min_child_weight': 0,
  'objective': 'reg:linear',
  'subsample': 1.0},
 {'colsample_bytree': 1.0,
  'eta': 0.01,

In [85]:
for i in range(0,7):
    model = xgb.train(params_list[i], 
                      dtrain_list[i], 
                      num_boost_round=num_boost_round,
                      evals=[(dtest_list[i], "Test")],
                      early_stopping_rounds=10)
    model_list.append(model)

[03:08:22] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[0]	Test-mae:18.39152
Will train until Test-mae hasn't improved in 10 rounds.
[1]	Test-mae:18.24244
[2]	Test-mae:18.09942
[3]	Test-mae:17.96146
[4]	Test-mae:17.81824
[5]	Test-mae:17.68406
[6]	Test-mae:17.54384
[7]	Test-mae:17.40943
[8]	Test-mae:17.27911
[9]	Test-mae:17.14693
[10]	Test-mae:17.01597
[11]	Test-mae:16.88744
[12]	Test-mae:16.76051
[13]	Test-mae:16.63345
[14]	Test-mae:16.51015
[15]	Test-mae:16.38758
[16]	Test-mae:16.26442
[17]	Test-mae:16.14717
[18]	Test-mae:16.02602
[19]	Test-mae:15.90732
[20]	Test-mae:15.79133
[21]	Test-mae:15.67836
[22]	Test-mae:15.56560
[23]	Test-mae:15.45357
[24]	Test-mae:15.34660
[25]	Test-mae:15.24339
[26]	Test-mae:15.13741
[27]	Test-mae:15.03040
[28]	Test-mae:14.92737
[29]	Test-mae:14.82348
[30]	Test-mae:14.72289
[31]	Test-mae:14.62310
[32]	Test-mae:14.52336
[33]	Test-mae:14.42561
[34]	Test-mae:14.33137
[35]	Test-mae:14.23874

[349]	Test-mae:7.61815
[350]	Test-mae:7.61573
[351]	Test-mae:7.61423
[352]	Test-mae:7.61299
[353]	Test-mae:7.61297
[354]	Test-mae:7.61172
[355]	Test-mae:7.61256
[356]	Test-mae:7.61212
[357]	Test-mae:7.61068
[358]	Test-mae:7.61023
[359]	Test-mae:7.61001
[360]	Test-mae:7.60895
[361]	Test-mae:7.60882
[362]	Test-mae:7.60749
[363]	Test-mae:7.60572
[364]	Test-mae:7.60488
[365]	Test-mae:7.60381
[366]	Test-mae:7.60309
[367]	Test-mae:7.60171
[368]	Test-mae:7.60114
[369]	Test-mae:7.60068
[370]	Test-mae:7.60019
[371]	Test-mae:7.59888
[372]	Test-mae:7.59820
[373]	Test-mae:7.59721
[374]	Test-mae:7.59649
[375]	Test-mae:7.59488
[376]	Test-mae:7.59493
[377]	Test-mae:7.59405
[378]	Test-mae:7.59430
[379]	Test-mae:7.59336
[380]	Test-mae:7.59373
[381]	Test-mae:7.59344
[382]	Test-mae:7.59339
[383]	Test-mae:7.59325
[384]	Test-mae:7.59361
[385]	Test-mae:7.59262
[386]	Test-mae:7.59106
[387]	Test-mae:7.59108
[388]	Test-mae:7.59138
[389]	Test-mae:7.59194
[390]	Test-mae:7.59077
[391]	Test-mae:7.59088
[392]	Test-

[189]	Test-mae:9.48894
[190]	Test-mae:9.46739
[191]	Test-mae:9.44704
[192]	Test-mae:9.42618
[193]	Test-mae:9.40652
[194]	Test-mae:9.38574
[195]	Test-mae:9.36591
[196]	Test-mae:9.34476
[197]	Test-mae:9.32499
[198]	Test-mae:9.30440
[199]	Test-mae:9.28463
[200]	Test-mae:9.26411
[201]	Test-mae:9.24514
[202]	Test-mae:9.22693
[203]	Test-mae:9.20894
[204]	Test-mae:9.18996
[205]	Test-mae:9.17016
[206]	Test-mae:9.15027
[207]	Test-mae:9.13126
[208]	Test-mae:9.11279
[209]	Test-mae:9.09429
[210]	Test-mae:9.07737
[211]	Test-mae:9.05939
[212]	Test-mae:9.04177
[213]	Test-mae:9.02566
[214]	Test-mae:9.00794
[215]	Test-mae:8.99047
[216]	Test-mae:8.97424
[217]	Test-mae:8.95812
[218]	Test-mae:8.94243
[219]	Test-mae:8.92629
[220]	Test-mae:8.90937
[221]	Test-mae:8.89270
[222]	Test-mae:8.87600
[223]	Test-mae:8.85971
[224]	Test-mae:8.84401
[225]	Test-mae:8.82888
[226]	Test-mae:8.81228
[227]	Test-mae:8.79705
[228]	Test-mae:8.78272
[229]	Test-mae:8.76771
[230]	Test-mae:8.75214
[231]	Test-mae:8.73607
[232]	Test-

[546]	Test-mae:7.35739
[547]	Test-mae:7.35743
[548]	Test-mae:7.35672
[549]	Test-mae:7.35653
[550]	Test-mae:7.35665
[551]	Test-mae:7.35657
[552]	Test-mae:7.35593
[553]	Test-mae:7.35611
[554]	Test-mae:7.35605
[555]	Test-mae:7.35658
[556]	Test-mae:7.35696
[557]	Test-mae:7.35640
[558]	Test-mae:7.35649
[559]	Test-mae:7.35704
[560]	Test-mae:7.35672
[561]	Test-mae:7.35616
[562]	Test-mae:7.35639
Stopping. Best iteration:
[552]	Test-mae:7.35593

[03:09:32] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[03:09:32] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[0]	Test-mae:17.09233
Will train until Test-mae hasn't improved in 10 rounds.
[1]	Test-mae:17.01905
[2]	Test-mae:16.94700
[3]	Test-mae:16.87445
[4]	Test-mae:16.80249
[5]	Test-mae:16.73606
[6]	Test-mae:16.66349
[7]	Test-mae:16.59222
[8]	Test-mae:16.52479
[9]	Test-mae:16.45536
[10]	Test-mae:16.38877
[11]	T

[322]	Test-mae:7.82311
[323]	Test-mae:7.81643
[324]	Test-mae:7.81005
[325]	Test-mae:7.80358
[326]	Test-mae:7.79752
[327]	Test-mae:7.79082
[328]	Test-mae:7.78431
[329]	Test-mae:7.77909
[330]	Test-mae:7.77237
[331]	Test-mae:7.76590
[332]	Test-mae:7.75935
[333]	Test-mae:7.75367
[334]	Test-mae:7.74797
[335]	Test-mae:7.74325
[336]	Test-mae:7.73773
[337]	Test-mae:7.73392
[338]	Test-mae:7.73032
[339]	Test-mae:7.72509
[340]	Test-mae:7.72053
[341]	Test-mae:7.71673
[342]	Test-mae:7.71257
[343]	Test-mae:7.70957
[344]	Test-mae:7.70516
[345]	Test-mae:7.70160
[346]	Test-mae:7.69813
[347]	Test-mae:7.69455
[348]	Test-mae:7.69036
[349]	Test-mae:7.68644
[350]	Test-mae:7.68261
[351]	Test-mae:7.67838
[352]	Test-mae:7.67609
[353]	Test-mae:7.67172
[354]	Test-mae:7.66889
[355]	Test-mae:7.66585
[356]	Test-mae:7.66259
[357]	Test-mae:7.65884
[358]	Test-mae:7.65568
[359]	Test-mae:7.65280
[360]	Test-mae:7.64976
[361]	Test-mae:7.64627
[362]	Test-mae:7.64295
[363]	Test-mae:7.63891
[364]	Test-mae:7.63676
[365]	Test-

...

[441]	Test-mae:6.73039
[442]	Test-mae:6.72957
[443]	Test-mae:6.72825
[444]	Test-mae:6.72692
[445]	Test-mae:6.72579
[446]	Test-mae:6.72502
[447]	Test-mae:6.72412
[448]	Test-mae:6.72340
[449]	Test-mae:6.72258
[450]	Test-mae:6.72192
[451]	Test-mae:6.72089
[452]	Test-mae:6.72021
[453]	Test-mae:6.71884
[454]	Test-mae:6.71818
[455]	Test-mae:6.71671
[456]	Test-mae:6.71586
[457]	Test-mae:6.71521
[458]	Test-mae:6.71496
[459]	Test-mae:6.71431
[460]	Test-mae:6.71325
[461]	Test-mae:6.71326
[462]	Test-mae:6.71238
[463]	Test-mae:6.71256
[464]	Test-mae:6.71230
[465]	Test-mae:6.71175
[466]	Test-mae:6.71101
[467]	Test-mae:6.71032
[468]	Test-mae:6.70979
[469]	Test-mae:6.70988
[470]	Test-mae:6.70979
[471]	Test-mae:6.70879
[472]	Test-mae:6.70789
[473]	Test-mae:6.70811
[474]	Test-mae:6.70838
[475]	Test-mae:6.70821
[476]	Test-mae:6.70816
[477]	Test-mae:6.70826
[478]	Test-mae:6.70812
[479]	Test-mae:6.70799
[480]	Test-mae:6.70746
[481]	Test-mae:6.70778
[482]	Test-mae:6.70805
[483]	Test-mae:6.70845
[484]	Test-

[291]	Test-mae:7.14956
[292]	Test-mae:7.14241
[293]	Test-mae:7.13677
[294]	Test-mae:7.13116
[295]	Test-mae:7.12540
[296]	Test-mae:7.11844
[297]	Test-mae:7.11174
[298]	Test-mae:7.10365
[299]	Test-mae:7.09629
[300]	Test-mae:7.08930
[301]	Test-mae:7.08265
[302]	Test-mae:7.07649
[303]	Test-mae:7.06956
[304]	Test-mae:7.06289
[305]	Test-mae:7.05742
[306]	Test-mae:7.05146
[307]	Test-mae:7.04513
[308]	Test-mae:7.04032
[309]	Test-mae:7.03625
[310]	Test-mae:7.03110
[311]	Test-mae:7.02544
[312]	Test-mae:7.01916
[313]	Test-mae:7.01271
[314]	Test-mae:7.00599
[315]	Test-mae:6.99925
[316]	Test-mae:6.99459
[317]	Test-mae:6.99005
[318]	Test-mae:6.98562
[319]	Test-mae:6.98024
[320]	Test-mae:6.97452
[321]	Test-mae:6.96845
[322]	Test-mae:6.96326
[323]	Test-mae:6.95913
[324]	Test-mae:6.95350
[325]	Test-mae:6.94756
[326]	Test-mae:6.94119
[327]	Test-mae:6.93598
[328]	Test-mae:6.92979
[329]	Test-mae:6.92380
[330]	Test-mae:6.91958
[331]	Test-mae:6.91419
[332]	Test-mae:6.90875
[333]	Test-mae:6.90353
[334]	Test-

[21]	Test-mae:15.39414
[22]	Test-mae:15.33579
[23]	Test-mae:15.27714
[24]	Test-mae:15.21916
[25]	Test-mae:15.16062
[26]	Test-mae:15.10353
[27]	Test-mae:15.04645
[28]	Test-mae:14.98960
[29]	Test-mae:14.93339
[30]	Test-mae:14.87742
[31]	Test-mae:14.82123
[32]	Test-mae:14.76626
[33]	Test-mae:14.71043
[34]	Test-mae:14.65555
[35]	Test-mae:14.60072
[36]	Test-mae:14.54715
[37]	Test-mae:14.49286
[38]	Test-mae:14.43832
[39]	Test-mae:14.38463
[40]	Test-mae:14.33065
[41]	Test-mae:14.27811
[42]	Test-mae:14.22500
[43]	Test-mae:14.17205
[44]	Test-mae:14.12026
[45]	Test-mae:14.06800
[46]	Test-mae:14.01666
[47]	Test-mae:13.96535
[48]	Test-mae:13.91416
[49]	Test-mae:13.86404
[50]	Test-mae:13.81387
[51]	Test-mae:13.76385
[52]	Test-mae:13.71511
[53]	Test-mae:13.66614
[54]	Test-mae:13.61737
[55]	Test-mae:13.56960
[56]	Test-mae:13.52197
[57]	Test-mae:13.47468
[58]	Test-mae:13.42707
[59]	Test-mae:13.38018
[60]	Test-mae:13.33453
[61]	Test-mae:13.28815
[62]	Test-mae:13.24140
[63]	Test-mae:13.19586
[64]	Test-m

[375]	Test-mae:7.58047
[376]	Test-mae:7.57734
[377]	Test-mae:7.57426
[378]	Test-mae:7.57151
[379]	Test-mae:7.56860
[380]	Test-mae:7.56533
[381]	Test-mae:7.56259
[382]	Test-mae:7.55992
[383]	Test-mae:7.55696
[384]	Test-mae:7.55418
[385]	Test-mae:7.55147
[386]	Test-mae:7.54871
[387]	Test-mae:7.54587
[388]	Test-mae:7.54384
[389]	Test-mae:7.54124
[390]	Test-mae:7.53850
[391]	Test-mae:7.53571
[392]	Test-mae:7.53295
[393]	Test-mae:7.53010
[394]	Test-mae:7.52750
[395]	Test-mae:7.52496
[396]	Test-mae:7.52216
[397]	Test-mae:7.52008
[398]	Test-mae:7.51767
[399]	Test-mae:7.51490
[400]	Test-mae:7.51238
[401]	Test-mae:7.51002
[402]	Test-mae:7.50731
[403]	Test-mae:7.50506
[404]	Test-mae:7.50299
[405]	Test-mae:7.50096
[406]	Test-mae:7.49790
[407]	Test-mae:7.49550
[408]	Test-mae:7.49313
[409]	Test-mae:7.49062
[410]	Test-mae:7.48869
[411]	Test-mae:7.48682
[412]	Test-mae:7.48438
[413]	Test-mae:7.48218
[414]	Test-mae:7.48028
[415]	Test-mae:7.47810
[416]	Test-mae:7.47601
[417]	Test-mae:7.47345
[418]	Test-

[78]	Test-mae:9.50503
[79]	Test-mae:9.45974
[80]	Test-mae:9.41635
[81]	Test-mae:9.37036
[82]	Test-mae:9.32726
[83]	Test-mae:9.28613
[84]	Test-mae:9.24440
[85]	Test-mae:9.20288
[86]	Test-mae:9.16351
[87]	Test-mae:9.12219
[88]	Test-mae:9.08465
[89]	Test-mae:9.04929
[90]	Test-mae:9.01083
[91]	Test-mae:8.97420
[92]	Test-mae:8.93981
[93]	Test-mae:8.90272
[94]	Test-mae:8.86790
[95]	Test-mae:8.83417
[96]	Test-mae:8.80138
[97]	Test-mae:8.76955
[98]	Test-mae:8.73724
[99]	Test-mae:8.70615
[100]	Test-mae:8.67377
[101]	Test-mae:8.64074
[102]	Test-mae:8.60852
[103]	Test-mae:8.57686
[104]	Test-mae:8.54575
[105]	Test-mae:8.51547
[106]	Test-mae:8.48660
[107]	Test-mae:8.45631
[108]	Test-mae:8.42821
[109]	Test-mae:8.39802
[110]	Test-mae:8.37154
[111]	Test-mae:8.34326
[112]	Test-mae:8.31447
[113]	Test-mae:8.28655
[114]	Test-mae:8.25995
[115]	Test-mae:8.23333
[116]	Test-mae:8.20870
[117]	Test-mae:8.18578
[118]	Test-mae:8.16419
[119]	Test-mae:8.14040
[120]	Test-mae:8.11907
[121]	Test-mae:8.09852
[122]	Test

In [88]:
mae_list = []
predict_list = []
for i in range(0,7):
    item = mean_absolute_error(model_list[i].predict(dtest_list[i]), y_test[i])
    mae_list.append(item)
    predict_list.append(model_list[i].predict(dtest_list[i]))

In [89]:
mae_list

[7.565343859177476,
 7.356392943981098,
 7.347447363510782,
 6.708913309365204,
 6.488322057264959,
 7.33650972855551,
 7.106610621555208]